In [39]:
import pandas as pd
import os

In [41]:
"""

Set to file path where CheXper-v1.0-small is located

For example, if your directory looks like '/Users/tperera/documents/XRayData/CheXper-v1.0-small', then set the path to '/Users/tperera/documents/XRayData'

"""

PATH_TO_CHEXPERT_DOWNLOADS = "C:/Users/samee/Downloads/Chexpert-Categorical-Single"

In [42]:
# read in chexpert_train.csv
chexpert_train = pd.read_csv('chexpert_train.csv')

In [43]:
# remove people with support devices

chexpert_train.drop(chexpert_train[chexpert_train['Support Devices'] == 1].index, inplace=True)

In [44]:
# remove children (under 18)
chexpert_train.drop(chexpert_train[chexpert_train['Age'] < 18].index, inplace=True)

In [45]:
cols = chexpert_train.columns[5:-1]
cols

Index(['No Finding', 'Enlarged Cardiomediastinum', 'Cardiomegaly',
       'Lung Opacity', 'Lung Lesion', 'Edema', 'Consolidation', 'Pneumonia',
       'Atelectasis', 'Pneumothorax', 'Pleural Effusion', 'Pleural Other',
       'Fracture'],
      dtype='object')

# The below cell removes patient data of patients with multiple conditions.

In [46]:
for index, row in chexpert_train.iterrows():
    numberOf1s = 0
    for column in cols:
        if row[column] == 1.0:
            numberOf1s += 1
    if numberOf1s > 1:
        chexpert_train.drop(index, inplace=True)
        # print("Dropped this row: ", index)

In [47]:
for index, row in chexpert_train.iterrows():
    if row['Frontal/Lateral'] == 'Lateral':
        chexpert_train.drop(index, inplace=True)

In [48]:
for index, row in chexpert_train[chexpert_train['AP/PA'] == 'LL'].iterrows():
    chexpert_train.drop(index, inplace=True)
chexpert_train.drop(201068, inplace=True)

In [51]:
# create all the folders

for disease in chexpert_train.columns[5:-1]:
    if not os.path.exists(os.path.join(PATH_TO_CHEXPERT_DOWNLOADS, disease)):
        os.makedirs(os.path.join(PATH_TO_CHEXPERT_DOWNLOADS, disease))

## Duplication strategy

In [52]:
for disease in chexpert_train.columns[5:-1]:
    imageNum = 0
    for file in chexpert_train[chexpert_train[disease] == 1.0]['Path'].to_list():
        imageNum += 1
        proj = chexpert_train[chexpert_train['Path'] == file]['AP/PA'].to_list()[0]
        os.rename(os.path.join(PATH_TO_CHEXPERT_DOWNLOADS, file), os.path.join(PATH_TO_CHEXPERT_DOWNLOADS, 'Train' + proj, disease, disease + ' ' + str(imageNum) + '.jpg'))